In [ ]:
import sys
sys.path.append('../src')

import os

In [ ]:
from read_data import Data
from analysis import analysis
from predictions import predict
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
model_path = '/users/rkakodka/Automated_analysis_of_zebrafish/testing_zebrafish_video-Rohit-2020-10-05/exported-models/DLC_testing_zebrafish_video_resnet_50_iteration-0_shuffle-1/snapshot-300000.pb'
image_folder = os.path.join(os.getcwd(), '..', 'data/96_well_plate/images/')
images = Data(image_folder + 'IMG_%04d.JPG')
experiment = analysis(images)
infer = predict(images, experiment, model_path)

In [ ]:
sess, output, input = infer.get_session()

In [ ]:
ret, frame, img_no = images.read()

In [ ]:
wells = experiment.detect_wells(R = [72, 108], image=frame)

In [ ]:
experiment.plot_wells(image = frame, wells = wells)

In [ ]:
def predict_image(image, wells, sess, input, output):

    well_ind, data = experiment.crop_wells(wells, image)
    predictions = sess.run(output, feed_dict = {input : data})
    
    predictions = np.asarray(predictions).reshape(3, len(wells), 3)
    predictions = (predictions.view([(f'f{i}',predictions.dtype) 
                                     for i in range(predictions.shape[-1])])[...,0].astype('O'))
    
    predicted_image = pd.DataFrame(predictions.transpose(), columns = ['right_eye', 'left_eye', 'yolk'], 
                                   index = pd.MultiIndex.from_tuples(well_ind))
    
    return predicted_image

predicted_image = predict_image(frame, wells, sess, input, output)

In [ ]:
%%time

predictions = infer.predict(wells = wells)

In [ ]:
predictions.head()

In [ ]:
predicted_image

In [ ]:
%%time

well_ind, data = experiment.crop_wells(wells, frame)

In [ ]:
%%time

predictions = sess.run(output, feed_dict = {input : data})

In [ ]:
%%time
predictions = np.asarray(predictions).reshape(3, len(wells), 3)
predictions = (predictions.view([(f'f{i}',predictions.dtype) 
                                     for i in range(predictions.shape[-1])])[...,0].astype('O'))
    
predicted_image = pd.DataFrame(predictions.transpose(), columns = ['right_eye', 'left_eye', 'yolk'], 
                                   index = pd.MultiIndex.from_tuples(well_ind))